In [ ]:
import json
import os
import requests
from typing import List, Dict, Any

CFBD_API_URL = "https://api.collegefootballdata.com/games"
try:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ is not defined (e.g., running in an interactive shell or notebook)
    BASE_DIR = os.getcwd()

CONFIG_PATH = os.path.join(BASE_DIR, "config.json")


def load_config(path: str = CONFIG_PATH) -> Dict[str, Any]:
    """
    Load configuration from a JSON file.

    Expected format (at minimum):

    {
      "CFBD_API_KEY": "your_real_key_here"
    }
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            cfg = json.load(f)
    except FileNotFoundError:
        raise RuntimeError(f"Config file not found at '{path}'. "
                           f"Create it with your CFBD_API_KEY.")
    except json.JSONDecodeError as e:
        raise RuntimeError(f"Error parsing JSON in config file '{path}': {e}")
    return cfg


config = load_config()
CFBD_API_KEY = config.get("CFBD_API_KEY")


if CFBD_API_KEY is None:
    raise RuntimeError("Set CFBD_API_KEY env var with your CollegeFootballData API key.")


# -------------------------------------------------------------------
# 1. You plug in your real SAMPLE_TEAMS list here (BYU must be team_id 1)
# -------------------------------------------------------------------

SAMPLE_TEAMS = [
    # ... paste your generated SAMPLE_TEAMS here ...
    # e.g. {"team_id": 1, "team_name": "BYU", "mascot": "Cougars", "conference": "Big 12"},
]

# Build quick lookup from team name to team_id
TEAM_NAME_TO_ID = {t["team_name"]: t["team_id"] for t in SAMPLE_TEAMS}


def fetch_2025_fbs_games() -> List[Dict[str, Any]]:
    """
    Call CFBD to get all completed 2025 FBS regular-season games.
    """
    headers = {
        "Authorization": f"Bearer {CFBD_API_KEY}",
    }
    params = {
        "year": 2025,
        "seasonType": "regular",
        "division": "fbs",
    }
    resp = requests.get(CFBD_API_URL, headers=headers, params=params, timeout=60)
    
    resp.raise_for_status()
    games = resp.json()
    
    # Filter out games without final scores (if any)
    games = [g for g in games if g.get("homePoints") is not None and g.get("awayPoints") is not None]
    
    return games


def map_team_name_to_id(team_name: str) -> int:
    """
    Map CFBD team name to your team_id.
    If you have slightly different naming, add manual overrides here.
    """
    # Direct match first
    if team_name in TEAM_NAME_TO_ID:
        return TEAM_NAME_TO_ID[team_name]

    # Example manual tweaks (add as needed):
    name_fixes = {
        # "Miami (FL)": "Miami",    # if you chose different canonical names
        # "UTSA": "UTSA",
    }
    if team_name in name_fixes and name_fixes[team_name] in TEAM_NAME_TO_ID:
        return TEAM_NAME_TO_ID[name_fixes[team_name]]

    raise KeyError(f"No team_id found for team_name '{team_name}'. "
                   f"Add it to SAMPLE_TEAMS or a name_fixes mapping.")


def build_sample_games(cfbd_games: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Convert CFBD game objects into your SAMPLE_GAMES format:

    {
        "game_id": int,
        "date": "YYYY-MM-DD",
        "home_team_id": int,
        "away_team_id": int,
        "home_score": int,
        "away_score": int,
        "neutral_site": 0/1,
    }
    """
    sample_games = []
    # Sort by kickoff date so game_id roughly follows time
    cfbd_games_sorted = sorted(cfbd_games, key=lambda g: g["startDate"])

    for i, g in enumerate(cfbd_games_sorted, start=1):
        home_team = g["home_team"]
        away_team = g["away_team"]

        try:
            home_id = map_team_name_to_id(home_team)
            away_id = map_team_name_to_id(away_team)
        except KeyError as e:
            # If you don't care about a few unmatched games, you can skip them here
            print(f"Skipping game due to unmatched team name: {e}")
            continue

        # CFBD uses ISO datetimes in start_date, e.g. "2025-08-30T17:00:00.000Z"
        date_str = g["startDate"][:10]  # "YYYY-MM-DD"

        neutral_flag = 1 if g.get("neutral_site") else 0

        sample_games.append({
            "game_id": i,
            "date": date_str,
            "home_team_id": home_id,
            "away_team_id": away_id,
            "home_score": int(g["home_points"]),
            "away_score": int(g["away_points"]),
            "neutral_site": neutral_flag,
        })

    return sample_games


def format_as_python_list(sample_games: List[Dict[str, Any]]) -> str:
    """
    Dump SAMPLE_GAMES as Python code.
    """
    lines = ["SAMPLE_GAMES = ["]
    for g in sample_games:
        line = (
            f'    {{"game_id": {g["game_id"]}, '
            f'"date": "{g["date"]}", '
            f'"home_team_id": {g["home_team_id"]}, '
            f'"away_team_id": {g["away_team_id"]}, '
            f'"home_score": {g["home_score"]}, '
            f'"away_score": {g["away_score"]}, '
            f'"neutral_site": {g["neutral_site"]}}},'
        )
        lines.append(line)
    lines.append("]")
    return "\n".join(lines)


def main():
    games = fetch_2025_fbs_games()
    
    sample_games = build_sample_games(games)
    print(f"# Fetched {len(sample_games)} completed FBS games for 2025\n")
    print(format_as_python_list(sample_games))


if __name__ == "__main__":
    main()
